<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/15_Finans_%C4%B0%C3%A7in_Python_RSI_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kütüphanelerin yüklenmesi
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener
!pip install backtesting


#kütüphanelerin çağırılması
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy
import numpy as np

#Tradingview Kütüphanesini açıyoruz.
tv = TvDatafeed()

#Hisse Adlarını Tanımlıyoruz.
Hisseler = ['AKBNK','ALARK','ASELS','ASTOR','BIMAS',
	'BRSAN','DOAS','EKGYO','ENKAI','EREGL',
	'FROTO','GARAN','GUBRF','HEKTS','ISCTR',
	'KCHOL','KONTR','KOZAL','KRDMD','OYAKC',
	'PETKM','PGSUS','SAHOL','SASA','SISE',
	'TCELL','THYAO','TOASO','TUPRS','YKBNK']


#Göreceli hareketli Ortalama
def rma(series, length=None):
    """
    Belirli bir kapanış fiyatı serisinin Göreceli Hareketli Ortalama'sını (RMA) hesaplar.
    """
    length = int(length) if length and length > 0 else 10
    alpha = (1.0 / length) if length > 0 else 0.5
    rma = series.ewm(alpha=alpha, min_periods=length).mean()
    return rma


#RSI - Göreceli Güç Endeksi
def rsi(series, length=14):
    """
    Belirli bir serinin Göreceli Güç Endeksi'ni (RSI) hesaplayın.

    Parametreler:
    - series: Fiyat verilerini içeren pandas Serisi.
    - length: RSI periyodunun uzunluğu (varsayılan 14'tür).
    - scalar: RSI değerlerini ayarlamak için kullanılan skaler faktör (varsayılan 100'dür).
    - drift: Fiyat değişiklikleri için dönem sayısı (varsayılan 1'dir).

    Döndürür:
    - Girdi parametrelerine göre hesaplanmış RSI değerlerini içeren pandas Serisi.
    """
    # Fiyat değişikliklerini hesaplayın
    scalar = 100
    drift = 1
    negative = series.diff(drift)
    positive = negative.copy()

    # Pozitif seri için negatifleri 0 yapın
    positive[positive < 0] = 0
    # Negatif seri için pozitifleri 0 yapın
    negative[negative > 0] = 0

    # Ortalama kazançları ve kayıpları hesaplayın
    positive_avg = rma(positive, length=length)
    negative_avg = rma(negative, length=length)

    # RSI'yi hesaplayın
    rsi = scalar * positive_avg / (positive_avg + negative_avg.abs())
    return rsi

def Sell_Strategy(data,tp=10,ec=0.90):
    df=data.copy()
    df['Stop Loss'] = np.nan
    df['Entry Price'] = np.nan
    df['Exit'] = False
    df['Trade'] = "BEKLE"
    in_trade=False
    for i in range(1,len(df)):
            if in_trade==False:
                entry_condition = (df.loc[i,'Entry'] == True) & (df.loc[i-1,'Entry']==False)
                if entry_condition:
                    in_trade = True
                    entry_price = df.loc[i, 'close']
                    stop_loss = entry_price*0.90

                    df.loc[i,'Entry'] = True
                    df.loc[i,'Trade'] = 'AL'
                    df.loc[i,'Entry Price']  = entry_price
                    df.loc[i,'Stop Loss'] = stop_loss
            else:
                exit_condition_1 = df.loc[i,'close'] < stop_loss
                exit_condition_2 = (df.loc[i-1, 'close'] > entry_price * (1 + tp / 100))
                if exit_condition_1:
                    in_trade = False
                    df.loc[i,'Exit'] = True
                    df.loc[i,'Trade'] = 'ZARAR KES'

                if exit_condition_2:
                    in_trade = False
                    df.loc[i,'Exit'] = True
                    df.loc[i,'Trade'] = 'SAT'
    return df

#Her Zaman kullanacağımız Stratejimizi hazırlıyoruz.
class Strategy(Strategy):
    def init(self):
        pass
    def next(self):
        if self.data['Entry'] == True and not self.position:
            self.buy()
        elif self.data['Exit'] == True:
            self.position.close()

# BIST30 sonuçları Arşivlemek için İki Adet Boş dataframe yapısı oluşturuyoruz.
df_bist30_RE = pd.DataFrame(columns=['Hisse Adı','Getiri Yüzdesi','Son Sinyal'])
df_bist30_WR = pd.DataFrame(columns=['Hisse Adı','Kazanma Oranı','Son Sinyal'])

for hisse in Hisseler:
  try:

    #Hisse Verilerini Tradingview Kütüphanesinden çekiyoruz.
    data = tv.get_hist(symbol=hisse, exchange='BIST', interval=Interval.in_daily, n_bars=5000)
    data.reset_index(inplace=True)

    #RSI i hesaplatıyoruz.
    data['RSI14'] = rsi(data['close'],14)
    data['Entry'] = (data['RSI14'] > 30)

    #Satış Stratejimizi devreye alıyoruz.
    data=Sell_Strategy(data,10,0.90)

    #data başlıklarını yeniden isimlendiriyoruz.
    data.rename(columns={'open': 'Open', 'high': 'High',
                        'low': 'Low',
                        'close': 'Close',
                        'volume': 'Volume'},
                        inplace=True)

    # Backtest yapımızı çağırıyoruz ve sonuçları inceliyoruz.
    bt = Backtest(data, Strategy, cash=100000, commission=0.002)
    Stats = bt.run()
    RE = round(Stats.loc['Return [%]'], 2)
    WR = round(Stats.loc['Win Rate [%]'], 2)

    #Son Sinyali Alıyoruz.
    Signals = data.tail(2).reset_index()
    Buy = Signals.loc[0, 'Entry'] == False and Signals.loc[1, 'Entry'] ==True

    df_bist30_RE.loc[len(df_bist30_RE)] = [hisse,RE,Buy]
    df_bist30_WR.loc[len(df_bist30_WR)] = [hisse,WR,Buy]
  except:
    pass

print(df_bist30_RE)
print(df_bist30_WR)

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-l7de37lq
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-l7de37lq
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


                datetime      symbol       open       high        low  \
0    2004-09-03 06:00:00  BIST:AKBNK   2.163530   2.180430   2.146630   
1    2004-09-06 06:00:00  BIST:AKBNK   2.146630   2.180430   2.129730   
2    2004-09-07 06:00:00  BIST:AKBNK   2.163530   2.197340   2.146630   
3    2004-09-08 06:00:00  BIST:AKBNK   2.180430   2.248040   2.163530   
4    2004-09-09 06:00:00  BIST:AKBNK   2.231140   2.248040   2.180430   
...                  ...         ...        ...        ...        ...   
4995 2024-07-29 06:00:00  BIST:AKBNK  64.699997  66.699997  64.500000   
4996 2024-07-30 06:00:00  BIST:AKBNK  64.500000  65.000000  62.500000   
4997 2024-07-31 06:00:00  BIST:AKBNK  62.700001  63.549999  61.450001   
4998 2024-08-01 06:00:00  BIST:AKBNK  62.900002  64.849998  62.849998   
4999 2024-08-02 06:00:00  BIST:AKBNK  62.000000  62.350000  59.700000   

          close      volume      RSI14  Entry  
0      2.163530   3742002.0        NaN  False  
1      2.146630   2487639.0

<ipython-input-12-41a61661fdaa>:142: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, Strategy, cash=100000, commission=0.002)


                datetime      symbol        open        high         low  \
0    2004-09-03 06:00:00  BIST:ALARK    1.374644    1.406613    1.363988   
1    2004-09-06 06:00:00  BIST:ALARK    1.406613    1.459893    1.395957   
2    2004-09-07 06:00:00  BIST:ALARK    1.459893    1.470550    1.438581   
3    2004-09-08 06:00:00  BIST:ALARK    1.459893    1.502518    1.438581   
4    2004-09-09 06:00:00  BIST:ALARK    1.502518    1.523830    1.491862   
...                  ...         ...         ...         ...         ...   
4995 2024-07-29 06:00:00  BIST:ALARK  102.800000  104.200000  102.000000   
4996 2024-07-30 06:00:00  BIST:ALARK  102.900000  106.800000  102.100000   
4997 2024-07-31 06:00:00  BIST:ALARK  105.300000  105.900000  103.400000   
4998 2024-08-01 06:00:00  BIST:ALARK  106.100000  108.000000  105.500000   
4999 2024-08-02 06:00:00  BIST:ALARK  106.800000  109.000000  104.500000   

           close     volume      RSI14  Entry  
0       1.406613  4829753.0        NaN 

<ipython-input-12-41a61661fdaa>:142: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, Strategy, cash=100000, commission=0.002)


...

<ipython-input-12-41a61661fdaa>:142: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, Strategy, cash=100000, commission=0.002)


                datetime      symbol       open       high        low  \
0    2004-08-26 06:00:00  BIST:TUPRS    1.38698    1.43741    1.37437   
1    2004-08-27 06:00:00  BIST:TUPRS    1.43741    1.46263    1.37437   
2    2004-08-31 06:00:00  BIST:TUPRS    1.43741    1.47524    1.42481   
3    2004-09-01 06:00:00  BIST:TUPRS    1.46263    1.48785    1.45002   
4    2004-09-02 06:00:00  BIST:TUPRS    1.47524    1.48785    1.45002   
...                  ...         ...        ...        ...        ...   
4995 2024-07-29 06:00:00  BIST:TUPRS  163.30000  164.39999  162.20000   
4996 2024-07-30 06:00:00  BIST:TUPRS  162.89999  164.30000  161.50000   
4997 2024-07-31 06:00:00  BIST:TUPRS  161.00000  163.89999  160.10001   
4998 2024-08-01 06:00:00  BIST:TUPRS  165.20000  166.70000  163.80000   
4999 2024-08-02 06:00:00  BIST:TUPRS  162.50000  163.00000  159.10000   

          close      volume      RSI14  Entry  
0       1.43741   5474467.0        NaN  False  
1       1.45002   6392693.0

<ipython-input-12-41a61661fdaa>:142: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, Strategy, cash=100000, commission=0.002)


                datetime      symbol       open       high        low  \
0    2004-08-27 06:00:00  BIST:YKBNK   0.788730   0.805390   0.783170   
1    2004-08-31 06:00:00  BIST:YKBNK   0.794280   0.810950   0.794280   
2    2004-09-01 06:00:00  BIST:YKBNK   0.805390   0.833160   0.805390   
3    2004-09-02 06:00:00  BIST:YKBNK   0.822050   0.827610   0.810950   
4    2004-09-03 06:00:00  BIST:YKBNK   0.816500   0.833160   0.816500   
...                  ...         ...        ...        ...        ...   
4995 2024-07-29 06:00:00  BIST:YKBNK  30.020000  30.680000  29.940001   
4996 2024-07-30 06:00:00  BIST:YKBNK  30.280001  30.580000  29.620001   
4997 2024-07-31 06:00:00  BIST:YKBNK  29.580000  30.379999  29.480000   
4998 2024-08-01 06:00:00  BIST:YKBNK  30.639999  32.320000  30.500000   
4999 2024-08-02 06:00:00  BIST:YKBNK  31.600000  32.040000  30.980000   

          close       volume      RSI14  Entry  
0      0.799840   53367976.0        NaN  False  
1      0.799840   7770754

<ipython-input-12-41a61661fdaa>:142: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, Strategy, cash=100000, commission=0.002)
